In [3]:
!python -m pip install obsws-python
!python -m pip install keyboard
!python -m pip install -U obsws-python

In [4]:
import time
from threading import Thread, Barrier
import obsws_python as obs

## Added imports for event logging via keyboard buttons and csv outputs
import csv
from datetime import datetime, timezone
from pathlib import Path
import keyboard
import threading

TARGETS = [ 
    # IP address, the websocket port, obs websocket password, name for printing messages
    # {"host": "10.0.0.137", "port": 4455, "password": "GARN_air", "name": "AIR COMPUTER"},
    ## {"host": "192.168.0.141", "port": 4454, "password": "GARN_air", "name": "AIR COMPUTER"},
    ## {"host": "10.0.0.52", "port": 4455, "password": "GARN_ground", "name": "GROUND COMPUTER"},
    {"host": "192.168.0.141", "port": 4454, "password": "GARN_ground_1", "name": "GROUND COMPUTER"},
]

## Added a mapping for keyboard buttons to event names
EVENT_KEYS = {
    "1": "Stimulus onset",
    "2": "Condition start",
    "3": "Condition end",
    "space": "Default manual marker",
}

## Added an exit key to end the logger
EXIT_KEYS = {"esc"}

## Name for the CSV file
LOG_PATH = Path("mdl_obs_events.csv")

# Function from Amanda
def start_record_job(tgt, go_barrier):
    # Requests client connection to OBS
    cl = obs.ReqClient(host=tgt["host"], port=tgt["port"], password=tgt["password"], timeout=3)
    
    # Asks OBS whether recording is already active
    try:
        status = cl.get_record_status()
        if getattr(status, "output_active", False):
            print(f'[{tgt["name"]}] Already recording.')
            return
        
        # The following ensures all OBS recordings start simultaneously
        go_barrier.wait()
        cl.start_record()
        
        # Wait for OBS to let OBS update its state
        time.sleep(0.5)
        
        # Asks again for status
        confirm = cl.get_record_status()
        if getattr(confirm, "output_active", False):
            print(f'[{tgt["name"]}] Recording started.')
        else:
            print(f'[{tgt["name"]}] Failed to start recording.')
            
    # error prints if anything goes wrong
    except Exception as e:
        print(f'[{tgt["name"]}] ERROR: {e}')
        
## Add function for keyboard button to stop the recording as well
def stop_record_job(tgt):
    ## Stops the reocrding on one OBS host
    cl = obs.ReqClient(host=tgt["host"], port=tgt["port"],
                       password=tgt["password"], timeout=3)
    name = tgt.get("name", f'{tgt["host"]}:{tgt["port"]}')
    try:
        status = cl.get_record_status()
        if not getattr(status, "output_active", False):
            print(f'[{name}] Not recording.')
            return

        cl.stop_record()

        # Poll up to ~5s until output_active becomes False
        for _ in range(50):
            time.sleep(0.1)
            confirm = cl.get_record_status()
            if not getattr(confirm, "output_active", False):
                print(f'[{name}] Recording stopped.')
                return

        print(f'[{name}] Stop requested; recorder still reporting active after timeout.')
    except Exception as e:
        print(f'[{name}] ERROR: {e}')

# Always overwrites the 
def init_csv_new(path: Path):
    """Always overwrite the CSV file with a fresh header row."""
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["Event Name", "Clock Time (UTC ISO)", "Elapsed Time (s)"])

def append_event(path: Path, event_name: str, start_iso: str, t0_perf: float):
    now_utc = datetime.now(timezone.utc)
    elapsed_s = time.perf_counter() - t0_perf
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow([event_name, now_utc.isoformat(), f"{elapsed_s:.3f}"])
    print(f'Logged: {event_name} | {now_utc.isoformat()} | {elapsed_s:.3f}s')

def run_keyboard_logger():
    init_csv_new(LOG_PATH)
    start_clock_iso = datetime.now(timezone.utc).isoformat()
    t0_perf = time.perf_counter()

    print("\nEvent logger active. Press keys to log events:")
    for k, name in EVENT_KEYS.items():
        print(f"  [{k}] -> {name}")
    print("Press [ESC] to stop logging only.")
    print("Press [Q]   to stop OBS recording (and logging).\n")

    # 1) Register event hotkeys
    event_hotkeys = []
    for key, event_name in EVENT_KEYS.items():
        h = keyboard.add_hotkey(
            key,
            lambda nm=event_name: append_event(LOG_PATH, nm, start_clock_iso, t0_perf)
            if logging_active.is_set() else None
        )
        event_hotkeys.append(h)

    # 2) Setup flags
    global logging_active
    logging_active = threading.Event()
    logging_active.set()  # start with logging enabled
    stop_and_stop_recording = threading.Event()

    # ESC stops logging (one-way)
    def stop_logging():
        if logging_active.is_set():
            logging_active.clear()
            print("Event logging stopped (recordings still running).")

    hk_esc = keyboard.add_hotkey("esc", stop_logging)

    # Q stops recording and exits
    hk_q = keyboard.add_hotkey("q", stop_and_stop_recording.set)

    # 3) Wait until Q is pressed
    while not stop_and_stop_recording.is_set():
        time.sleep(0.1)

    if logging_active.is_set():
        print("Stopping OBS recordings and logger...")
    else:
        print("Stopping OBS recordings...")

    # Stop all OBS recordings
    threads = [Thread(target=stop_record_job, args=(t,), daemon=True) for t in TARGETS]
    for th in threads:
        th.start()
    for th in threads:
        th.join()

    # 4) Cleanup
    for h in event_hotkeys:
        keyboard.remove_hotkey(h)
    keyboard.remove_hotkey(hk_esc)
    keyboard.remove_hotkey(hk_q)
    print("All done.\n")

def main():
    # Early exit if no OBS instances
    if not TARGETS:
        print("No targets configured.")
        return
    
    # Threads wait for all OBS instances on the network
    barrier = Barrier(len(TARGETS))
    
    # Creates on thread per OBS computer & starts recording
    threads = [Thread(target=start_record_job, args=(t, barrier), daemon=True) for t in TARGETS]
    
    # Start all threads
    for th in threads: th.start()
    for th in threads: th.join()
    
    ## Start keyboard based event logger
    run_keyboard_logger()

main()

[GROUND COMPUTER] Already recording.

Event logger active. Press keys to log events:
  [1] -> Stimulus onset
  [2] -> Condition start
  [3] -> Condition end
  [space] -> Default manual marker
Press [ESC] to stop logging only.
Press [Q]   to stop OBS recording (and logging).

Logged: Stimulus onset | 2025-08-29T14:35:58.942308+00:00 | 1.092s
Logged: Stimulus onset | 2025-08-29T14:35:59.823306+00:00 | 1.973s
Logged: Condition start | 2025-08-29T14:36:00.590228+00:00 | 2.740s
Logged: Condition start | 2025-08-29T14:36:00.842103+00:00 | 2.992s
Logged: Condition end | 2025-08-29T14:36:01.699929+00:00 | 3.849s
Logged: Condition end | 2025-08-29T14:36:02.660963+00:00 | 4.810s
Logged: Condition start | 2025-08-29T14:36:03.351825+00:00 | 5.501s
Logged: Stimulus onset | 2025-08-29T14:36:03.643017+00:00 | 5.792s
Logged: Stimulus onset | 2025-08-29T14:36:05.986616+00:00 | 8.136s
Logged: Condition start | 2025-08-29T14:36:06.565009+00:00 | 8.714s
Logged: Condition end | 2025-08-29T14:36:07.664072+0